In [ ]:
from flask import Flask, jsonify
import random
import time
import threading

app = Flask(__name__)

# Словарь для хранения состояния выполнения для каждого этапа
progress = {
    'setup_ai_parser': 0,
    'setup_ai_ranking': 0,
    'setup_ai_summarization': 0,
    'site_list': 0,
    'news_list': 0,
    'ranked_list': 0,
    'summary_list': 0,
    'digest': 0,
    'email': 0,
}

# Функция, эмулирующая выполнение пайплайна
def run_pipeline_step(step):
    for i in range(101):
        progress[step] = i
        time.sleep(random.uniform(0.01, 0.1))  # Эмуляция времени выполнения
    progress[step] = 100

@app.route('/start/<step>', methods=['GET'])
def start_pipeline_step(step):
    if step not in progress:
        return jsonify({"error": "Invalid step"}), 400

    # Запускаем выполнение в отдельном потоке, чтобы не блокировать основной поток
    thread = threading.Thread(target=run_pipeline_step, args=(step,))
    thread.start()
    
    return jsonify({"status": "Started", "step": step}), 200

@app.route('/progress/<step>', methods=['GET'])
def get_progress(step):
    if step not in progress:
        return jsonify({"error": "Invalid step"}), 400
    
    return jsonify({"step": step, "progress": progress[step]}), 200

if __name__ == '__main__':
    app.run(host='45.133.178.134', port=35474)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://45.133.178.134:35474
Press CTRL+C to quit
185.155.16.157 - - [11/Jun/2024 10:29:10] "GET /progress/setup_ai_parser HTTP/1.1" 200 -
185.155.16.157 - - [11/Jun/2024 10:29:14] "GET /progress/setup_ai_parser HTTP/1.1" 200 -
172.56.178.224 - - [12/Jun/2024 13:33:47] code 400, message Bad request version ('\x19íë\x16ÏWr.ÃW\x06:')
172.56.178.224 - - [12/Jun/2024 13:33:47] "\x16\x03\x01\x06à\x01\x00\x06Ü\x03\x03Q\x94\x07½ø§Áµ+¥Ûcóýk\x9f¦\x07Ùª².ô;bïV,ðæ®* ôà\x8c\x8aÒÀCÒ\x94¾%®Ú\x8aÅ¸t\x8aV\x15ô4«i:\x81\x0bóü\x16++\x00 ÚÚ\x13\x01\x13\x02\x13\x03À+À/À,À0Ì©Ì¨À\x13À\x14\x00\x9c\x00\x9d\x00/\x005\x01\x00\x06s\x9a\x9a\x00\x00\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\x0b\x00\x02\x01\x00þ\x0d\x00ú\x00\x00\x01\x00\x01\x80\x00 w_ß\x01D¾2Â^ª>ÆÜØo\x84a\x8b©\x99 É\x85<\x94Ñ\x00\x1fT\x15W\x1b\x00Ð\x1aNvAäèÕÝã½ïÛö¨\x9co\\ºO\x90\x0c\x19\x84gR2\x94¶ó\x07ê(&v) Àc[ê\x16fÓ\\\x0by&SÏBu\x94(ºdqÌþíy\x9f\x97@\x94\x83Åª`²øm¯û ²å\x04Á\x96\x88Kµë\x80AZLÓ\x8e1\x19\x10\x87·\\í¥{Þ$\x80\x189"Ð\x82\x0c\x1

In [ ]:
!pip install flask